In [1]:
# Cell 1: Mount Google Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Cell 2: Install dependencies (run once)
!pip install -q streamlit pyngrok joblib pandas scikit-learn


In [3]:
# Cell 3: Copy model & preprocessing files from Drive to a local folder
import os, shutil, sys

# <-- EDIT this path to the folder in your Drive where the .pkl files are stored
DRIVE_BASE = "/content/drive/MyDrive/DEEP LEARNING NOTEBOOKS/SUPERVISED MACHINE LEARNING/CLASSIFICATION ALGORITHMS/TITANIC DATA CLASSIFICATION/TitanicModel&Scaler"

# Filenames (update names if your files are named differently)
MODEL_NAME = "Titanic_pred_model.pkl"
SCALER_NAME = "titanic_scaler.pkl"
COLUMNS_NAME = "model_columns.pkl"

SRC_FILES = {
    "model": os.path.join(DRIVE_BASE, MODEL_NAME),
    "scaler": os.path.join(DRIVE_BASE, SCALER_NAME),
    "columns": os.path.join(DRIVE_BASE, COLUMNS_NAME)
}

TARGET_DIR = "/content/app_artifacts"
os.makedirs(TARGET_DIR, exist_ok=True)

for k, src in SRC_FILES.items():
    if os.path.exists(src):
        shutil.copy(src, TARGET_DIR)
        print(f"Copied {src} -> {TARGET_DIR}")
    else:
        print(f"ERROR: file not found: {src}")


Copied /content/drive/MyDrive/DEEP LEARNING NOTEBOOKS/SUPERVISED MACHINE LEARNING/CLASSIFICATION ALGORITHMS/TITANIC DATA CLASSIFICATION/TitanicModel&Scaler/Titanic_pred_model.pkl -> /content/app_artifacts
Copied /content/drive/MyDrive/DEEP LEARNING NOTEBOOKS/SUPERVISED MACHINE LEARNING/CLASSIFICATION ALGORITHMS/TITANIC DATA CLASSIFICATION/TitanicModel&Scaler/titanic_scaler.pkl -> /content/app_artifacts
Copied /content/drive/MyDrive/DEEP LEARNING NOTEBOOKS/SUPERVISED MACHINE LEARNING/CLASSIFICATION ALGORITHMS/TITANIC DATA CLASSIFICATION/TitanicModel&Scaler/model_columns.pkl -> /content/app_artifacts


In [4]:
# Cell 4: confirm you can load the .pkl files (local copies)
import joblib, os
MODEL_PATH = "/content/app_artifacts/" + MODEL_NAME
SCALER_PATH = "/content/app_artifacts/" + SCALER_NAME
COLUMNS_PATH = "/content/app_artifacts/" + COLUMNS_NAME

print("Files exist:",
      os.path.exists(MODEL_PATH),
      os.path.exists(SCALER_PATH),
      os.path.exists(COLUMNS_PATH))

# Try loading (will raise if incompatible)
model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
model_columns = joblib.load(COLUMNS_PATH)
print("Model, scaler, columns loaded successfully.")


Files exist: True True True
Model, scaler, columns loaded successfully.


In [5]:
# Cell 5: Create the Streamlit app file (app.py)
app_code = r'''
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os

# Paths (these point to the copied artifacts in the Colab filesystem)
BASE = "/content/app_artifacts"
MODEL_PATH = os.path.join(BASE, "''' + MODEL_NAME + r'''")
SCALER_PATH = os.path.join(BASE, "''' + SCALER_NAME + r'''")
COLUMNS_PATH = os.path.join(BASE, "''' + COLUMNS_NAME + r'''")

# Load artifacts lazily (so Streamlit's reruns are quicker)
@st.cache_resource
def load_artifacts():
    model = joblib.load(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    columns = joblib.load(COLUMNS_PATH)
    return model, scaler, columns

model, scaler, model_columns = load_artifacts()

st.set_page_config(page_title="Titanic Survival Predictor", layout="centered")
st.title("🚢 Titanic Survival Prediction")

st.markdown("Enter passenger details and click **Predict Survival**")

with st.form("input_form"):
    pclass = st.selectbox("Passenger Class (1 = 1st, 2 = 2nd, 3 = 3rd)", [1, 2, 3], index=2)
    sex = st.selectbox("Sex", ["male", "female"])
    age = st.number_input("Age", min_value=0, max_value=120, value=25)
    sibsp = st.number_input("Number of Siblings/Spouses Aboard", min_value=0, max_value=10, value=0)
    parch = st.number_input("Number of Parents/Children Aboard", min_value=0, max_value=10, value=0)
    fare = st.number_input("Passenger Fare", min_value=0.0, max_value=1000.0, value=32.2)
    embarked = st.selectbox("Port of Embarkation", ["C", "Q", "S"])
    submit = st.form_submit_button("Predict Survival")

if submit:
    sex_val = 1 if sex == "male" else 0
    embarked_map = {"C": 0, "Q": 1, "S": 2}
    embarked_val = embarked_map[embarked]
    # Build input row using saved model columns
    try:
        X = pd.DataFrame([[pclass, sex_val, age, sibsp, parch, fare, embarked_val]],
                         columns=model_columns)
    except Exception as e:
        st.error("Error constructing input DataFrame with model columns. See details:")
        st.exception(e)
    else:
        X_scaled = scaler.transform(X)
        pred = model.predict(X_scaled)
        proba = None
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X_scaled)[0].max()
        label = "Survived 💚" if int(pred[0]) == 1 else "Did Not Survive 💔"
        st.subheader(f"Prediction: {label}")
        if proba is not None:
            st.write(f"Confidence: {proba:.2%}")
'''
# write to file
with open("/content/app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print("Wrote /content/app.py")


Wrote /content/app.py


In [6]:
from pyngrok import ngrok
ngrok.set_auth_token("3568AqlgNHKdT8UTS6mXZ4yzIvG_wk8dirrAAZph98vZccL")

In [7]:
public_url = ngrok.connect(8501)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://unvindicable-corrie-centum.ngrok-free.dev" -> "http://localhost:8501"


In [8]:
# Cell 6: Start ngrok and run Streamlit (Colab) — Be careful with token exposure
#from pyngrok import ngrok
import time, os, signal, subprocess

# NGROK_TOKEN = "3568AqlgNHKdT8UTS6mXZ4yzIvG_wk8dirrAAZph98vZccL"   # <-- replace carefully, or leave empty to skip creating tunnel

# if NGROK_TOKEN:
#     ngrok.set_auth_token(NGROK_TOKEN)
#     public_url = ngrok.connect(8501)
#     print("Ngrok tunnel created:", public_url)

# Start Streamlit in the background and print log tail
cmd = ["streamlit", "run", "/content/app.py", "--server.port=8501", "--server.headless=true"]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Wait a couple seconds then show the first lines of output
time.sleep(3)
print("Streamlit started — logs (first 10 lines):")
for _ in range(10):
    line = proc.stdout.readline()
    if not line:
        break
    print(line.strip())

print("\nIf ngrok token provided, access via the printed public URL. Otherwise, use the local URL in the cell output.")


Streamlit started — logs (first 10 lines):



You can now view your Streamlit app in your browser.

Local URL: http://localhost:8501
Network URL: http://172.28.0.12:8501
External URL: http://34.68.78.226:8501


If ngrok token provided, access via the printed public URL. Otherwise, use the local URL in the cell output.


In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os
from datetime import datetime


In [9]:
import shutil

shutil.make_archive("/content/TitanicModel_Files", 'zip', "/content/drive/MyDrive/DEEP LEARNING NOTEBOOKS/SUPERVISED MACHINE LEARNING/CLASSIFICATION ALGORITHMS/TITANIC DATA CLASSIFICATION/TitanicModel&Scaler")


'/content/TitanicModel_Files.zip'